In [2]:
%load_ext tensorboard

In [3]:
!pip install x-unet
!pip install scikit-learn
!pip install scikit-image

DEPRECATION: Loading egg at /users/kolodny/gyanai/miniconda3/lib/python3.13/site-packages/CLEAN-0.1-py3.13.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
DEPRECATION: Loading egg at /users/kolodny/gyanai/miniconda3/lib/python3.13/site-packages/CLEAN-0.1-py3.13.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
DEPRECATION: Loading egg at /users/kolodny/gyanai/miniconda3/lib/python3.13/site-packages/CLEAN-0.1-py3.13.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can b

In [4]:
import random
import torch

def set_seed(seed: int = 0):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

In [5]:
set_seed()

In [6]:
import os

data_path = '/users/kolodny/gyanai/LF-MRI-Denoising/Data'
png_folder = os.path.join(data_path, 'Images')

checkpoints_path = '/users/kolodny/gyanai/LF-MRI-Denoising/checkpoints'
os.makedirs(checkpoints_path, exist_ok=True)

In [7]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from PIL import Image

class HighFrequencyDataset(Dataset):
    def __init__(self, images_path):
        self.images_path = images_path
        self.images = self.read_images()

    def read_images(self) -> torch.Tensor:
        image_file_paths = [f for f in os.listdir(self.images_path) if f.lower().endswith('.png')]
        images = [Image.open(os.path.join(self.images_path, img_path)) for img_path in image_file_paths]
        return self.transform_images(images)

    def transform_images(self, images: list[Image]) -> torch.Tensor:
        transform = T.Compose([
            T.Resize((256, 256)),
            T.ToTensor(),
        ])
        return torch.stack([transform(image) for image in images])

    def __len__(self) -> int:
        return len(self.images)

    def __getitem__(self, idx) -> torch.Tensor:
        return self.images[idx]


In [8]:
import torch


def add_gaussian_noise(x: torch.Tensor, max_sigma: float, min_sigma: float = 0):
    sigmas = torch.rand(x.size(0), 1, 1, 1, device=x.device) * (max_sigma - min_sigma) + min_sigma
    noise = torch.randn_like(x) * sigmas
    return x + noise, sigmas

def add_rician_noise(img: torch.Tensor, max_sigma: float, min_sigma: float = 0) -> tuple[torch.Tensor, float]:
    """
    Add Rician noise to an image tensor.

    Args:
        img (torch.Tensor): Input image (float tensor).
        sigma (float): Standard deviation of the underlying Gaussian noise.

    Returns:
        torch.Tensor: Noisy image with Rician-distributed values.
    """
    sigma = torch.rand(1).item() * (max_sigma - min_sigma) + min_sigma
    # Generate Gaussian noise with mean=0, std=sigma
    n1 = torch.normal(mean=0.0, std=sigma, size=img.shape, device=img.device)
    n2 = torch.normal(mean=0.0, std=sigma, size=img.shape, device=img.device)

    # Apply Rician formula
    noisy_img = torch.sqrt((img + n1) ** 2 + n2 ** 2)

    return noisy_img, sigma

# def add_rician_noise(img: torch.Tensor, max_sigma: float, min_sigma: float = 0):
#     sigma = np.random.uniform(min_sigma, max_sigma)
#     n1 = np.random.normal(0, sigma, img.shape)
#     n2 = np.random.normal(0, sigma, img.shape)
#     return np.sqrt((img + n1)**2 + n2**2)

In [9]:
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

def compute_psnr(clean_image: torch.Tensor, denoised_image: torch.Tensor):
    clean_image = clean_image.cpu().numpy()
    denoised_image = denoised_image.cpu().numpy()

    return psnr(clean_image, denoised_image, data_range=1.0)

def compute_ssim(clean_image: torch.Tensor, denoised_image: torch.Tensor):
    clean_image = clean_image.cpu().numpy()
    denoised_image = denoised_image.cpu().numpy()

    return ssim(clean_image, denoised_image, data_range=1.0)

In [10]:
dataset = HighFrequencyDataset(png_folder)

In [11]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

train_dataset, val_dataset = train_test_split(dataset, test_size=0.2, random_state=0)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [12]:
print(len(train_dataset))

26006


In [13]:
import torch
import torch.nn as nn
from x_unet import XUnet

# Create an instance of the XUnet model
unet = XUnet(
    dim=64,
    channels=1,
    dim_mults=(1, 2, 4, 8),
    nested_unet_depths=(7, 4, 2, 1),  # nested unet depths, from unet-squared paper
    consolidate_upsample_fmaps=True,  # whether to consolidate outputs from all upsample blocks, used in unet-squared paper
)

unet = nn.Sequential(unet, nn.Sigmoid())

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
unet = unet.to(device)

In [15]:
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-4

# Define optimizer
opt = torch.optim.AdamW(unet.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

In [16]:
import torch.nn as nn

# Define the loss function
mse_loss = nn.MSELoss()

In [ ]:
from tqdm import tqdm
import torch.nn.functional as F

NOISE_STD = 0.08
EPOCHS = 10
MODEL_OUT_DIR = os.path.join(checkpoints_path, 'vanilla')

os.makedirs(MODEL_OUT_DIR, exist_ok=True)


best_val = float("inf")

for epoch in range(1, EPOCHS + 1):
    unet.train()
    pbar = tqdm(iter(train_loader), desc=f"Epoch {epoch}/{EPOCHS}")
    
    train_loss = 0.0

    for x in pbar:
        x = x.to(device)
        x_noisy, _ = add_gaussian_noise(x, NOISE_STD)
        pred_noise = unet(x_noisy)          # predict clean image directly
        pred = x_noisy - pred_noise
        loss = mse_loss(pred, x)     # MSE loss
        loss.backward()
        opt.step()
        opt.zero_grad()

        train_loss += loss.item()

        pbar.set_postfix(loss=f"{loss.item():.4f}")

    train_loss /= len(train_loader)
    tqdm.write(f"Epoch {epoch} | train {train_loss:.8f}")

    # validation
    unet.eval()
    val_loss = 0.0
    with torch.no_grad():
        for xv in iter(val_loader):
            xv = xv.to(device)
            xv_noisy, _ = add_gaussian_noise(xv, NOISE_STD)
            pred_noise = unet(xv_noisy)
            pred = xv_noisy - pred_noise
            val_loss += mse_loss(pred, xv).item()
    val_loss /= len(val_loader)

    # track best
    if val_loss < best_val:
        best_val = val_loss
        torch.save(unet.state_dict(), os.path.join(MODEL_OUT_DIR, "best_model.pt"))

    tqdm.write(f"Epoch {epoch} | val {val_loss:.8f} | best {best_val:.8f}")

Epoch 1/10: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1626/1626 [14:11<00:00,  1.91it/s, loss=0.0019]


Epoch 1 | train 0.00224864
Epoch 1 | val 0.00212921 | best 0.00212921


Epoch 2/10: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1626/1626 [14:11<00:00,  1.91it/s, loss=0.0021]


Epoch 2 | train 0.00212937
Epoch 2 | val 0.00212261 | best 0.00212261


Epoch 3/10: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1626/1626 [14:10<00:00,  1.91it/s, loss=0.0031]


...

Epoch 6/10: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1626/1626 [14:10<00:00,  1.91it/s, loss=0.0022]


Epoch 6 | train 0.00212449
Epoch 6 | val 0.00213701 | best 0.00212261


Epoch 7/10: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1626/1626 [14:10<00:00,  1.91it/s, loss=0.0018]


Epoch 7 | train 0.00213074
Epoch 7 | val 0.00217108 | best 0.00212261


Epoch 8/10:  84%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                      | 1358/1626 [11:50<02:20,  1.91it/s, loss=0.0024]

In [ ]:
import matplotlib.pyplot as plt

_ = unet.eval()

In [ ]:
vd = torch.concat(val_dataset).unsqueeze(1)

random_indices = torch.randint(
    0, len(vd), (5,)
)  # just to make sure the model is loaded correctly
plt.figure(figsize=(15, 25))

with torch.no_grad():
    for i, xv in enumerate(vd[random_indices]):
        xv = xv.to(device).unsqueeze(0)
        xv_noisy, sigma = add_gaussian_noise(xv, 0.08, 0.03)
        pred_noise = unet(xv_noisy)
        pred = (xv_noisy - pred_noise).clamp(0, 1)
        clamped_noisy = xv_noisy.clamp(0, 1)

        plt.subplot(5, 3, i * 3 + 1)
        plt.imshow(xv.cpu().numpy().squeeze(), cmap="gray")
        plt.title("Original")
        plt.axis("off")

        plt.subplot(5, 3, i * 3 + 2)
        plt.imshow(clamped_noisy.cpu().numpy().squeeze(), cmap="gray")
        plt.title(
            f"Noisy\nsigma: {sigma.item():.4f}\n(psnr: {compute_psnr(xv.squeeze(), clamped_noisy.squeeze()):.4f}, ssim: {compute_ssim(xv.squeeze(), clamped_noisy.squeeze()):.4f})",
            wrap=True,
        )
        plt.axis("off")

        plt.subplot(5, 3, i * 3 + 3)
        plt.imshow(pred.cpu().numpy().squeeze(), cmap="gray")
        plt.title(
            f"Denoised\n(psnr: {compute_psnr(xv.squeeze(), pred.squeeze()):.4f}, ssim: {compute_ssim(xv.squeeze(), pred.squeeze()):.4f})",
            wrap=True,
        )
        plt.axis("off")

plt.tight_layout()
plt.show()

NameError: name 'torch' is not defined

In [ ]:
validation_loss = []
validation_psnr = []
validation_ssim = []

for xv in tqdm(vd):
    with torch.no_grad():
        xv = xv.to(device).unsqueeze(0)
        xv_noisy, sigma = add_gaussian_noise(xv, 0.08, 0.03)
        pred_noise = unet(xv_noisy)
        pred = (xv_noisy - pred_noise).clamp(0, 1)

    validation_loss.append(mse_loss(pred, xv).item())
    validation_psnr.append(compute_psnr(xv.squeeze(), pred.squeeze()))
    validation_ssim.append(compute_ssim(xv.squeeze(), pred.squeeze()))

print(f"Validation Loss: {sum(validation_loss)/len(validation_loss)}")
print(f"Validation PSNR: {sum(validation_psnr)/len(validation_psnr)}")
print(f"Validation SSIM: {sum(validation_ssim)/len(validation_ssim)}")

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6502/6502 [05:05<00:00, 21.26it/s]

Validation Loss: 0.0023875821988717533
Validation PSNR: 26.848815629173597
Validation SSIM: 0.48496769629742076
